In [ ]:
https://www.kaggle.com/code/taronzakaryan/predicting-stock-price-using-lstm-model-pytorch

In [1]:
import torch
import torch.nn as nn
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv('data/EURUSD/M15.csv', index_col=False)
df = df.drop(columns=['time'])

In [3]:
#df = pd.read_csv('data/EURUSD/M15.csv', index_col=False)
target = pd.DataFrame(df.close).pct_change()
target = target > 0
target = target.replace(True, 1)
target = target.replace(False, 0)
#removing the first line because we use pct_change 
df = df[1:]
target = target[1:]

In [4]:
window_size = 25
n = np.array(df)
x = np.array([n[i-window_size:i] for i in range(window_size, len(df))], dtype=object)
#df = torch.from_numpy(x.astype(np.float32))
df = x.astype(np.float32)
target = target[window_size:]

In [5]:
train_size = int(len(df)*0.8)
x_train = df[:train_size]
y_train = np.array(target[:train_size]).astype(np.float32)
x_test = df[train_size:]
y_test =  np.array(target[train_size:]).astype(np.float32)

In [6]:
print(f'Shape of x_train is {x_train.shape}')
print(f'Shape of x_test is {x_test.shape}')
print(f'Shape of y_train is {y_train.shape}')
print(f'Shape of y_test is {y_test.shape}')

Shape of x_train is (47979, 25, 91)
Shape of x_test is (11995, 25, 91)
Shape of y_train is (47979, 1)
Shape of y_test is (11995, 1)


In [7]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)
y_train.size(),x_train.size()

(torch.Size([47979, 1]), torch.Size([47979, 25, 91]))

In [8]:
sequence_length = 28
input_size = 91
hidden_size = 128
num_layers = 4
num_classes = 2
batch_size = 1000
num_epochs = 5
learning_rate = 0.001

In [9]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, hidden = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :])
        return out
        
        
model = RNN(input_size, hidden_size, num_layers, num_classes)
print(model)
     

RNN(
  (lstm): LSTM(91, 128, num_layers=4, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


In [10]:
loss_func = nn.CrossEntropyLoss()
from torch import optim
optimizer = optim.Adam(model.parameters(), lr = 0.001) 
     

In [11]:
def train(num_epochs, model):
        
    # Train the model
    total_step = len(x_train)
    correct_preds = 0
    for epoch in range(num_epochs):
        print("-"*50,f' Epoch {epoch+1} ',"-"*50)
        for i in range(total_step):
            
            # Forward pass
            outputs = model((x_train[i].reshape(-1,x_train[i].size(0) , x_train[i].size(1))))
            outputs = outputs.float()
            if y_train[i].item() == 1:
                target = torch.tensor([1]).long()
            elif y_train[i].item() == 0:
                target = torch.tensor([0]).long()
            loss = loss_func(outputs, target)
            correct_preds += target.eq(outputs.argmax()).sum().item()
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % batch_size == 0:
                accuracy = correct_preds / batch_size
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.2f} | Accuracy: {:.2f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), accuracy))
                correct_preds = 0
                


train(num_epochs, model)

--------------------------------------------------  Epoch 1  --------------------------------------------------
Epoch [1/5], Step [1000/47979], Loss: 0.74 | Accuracy: 0.51
Epoch [1/5], Step [2000/47979], Loss: 0.70 | Accuracy: 0.48
Epoch [1/5], Step [3000/47979], Loss: 0.68 | Accuracy: 0.49
Epoch [1/5], Step [4000/47979], Loss: 0.68 | Accuracy: 0.49
Epoch [1/5], Step [5000/47979], Loss: 0.69 | Accuracy: 0.47
Epoch [1/5], Step [6000/47979], Loss: 0.71 | Accuracy: 0.49
Epoch [1/5], Step [7000/47979], Loss: 0.67 | Accuracy: 0.52
Epoch [1/5], Step [8000/47979], Loss: 0.66 | Accuracy: 0.50
Epoch [1/5], Step [9000/47979], Loss: 0.74 | Accuracy: 0.52
Epoch [1/5], Step [10000/47979], Loss: 0.75 | Accuracy: 0.53
Epoch [1/5], Step [11000/47979], Loss: 0.67 | Accuracy: 0.50
Epoch [1/5], Step [12000/47979], Loss: 0.71 | Accuracy: 0.51
Epoch [1/5], Step [13000/47979], Loss: 0.69 | Accuracy: 0.47
Epoch [1/5], Step [14000/47979], Loss: 0.71 | Accuracy: 0.49
Epoch [1/5], Step [15000/47979], Loss: 0.71